In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [10]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [11]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset=dataset.astype(int)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27,0,16884,0,1
1,18,33,1,1725,1,0
2,28,33,3,4449,1,0
3,33,22,0,21984,1,0
4,32,28,0,3866,1,0
...,...,...,...,...,...,...
1333,50,30,3,10600,1,0
1334,18,31,0,2205,0,0
1335,18,36,0,1629,0,0
1336,21,25,0,2007,0,0


In [12]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [13]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
dependent=dataset[['charges']]

In [31]:
!pip install --upgrade numpy

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

#from sklearn.tree import DecisionTreeRegressor
param_grid = {'criterion':['mse','mae'],
            'max_features': ['auto','sqrt','log2'],
            'n_estimators':[10,100],}


grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose= 3,n_jobs=-1, scoring='neg_mean_squared_error',cv=5)

# fitting the model for grid search
grid.fit(independent, dependent)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='neg_mean_squared_error', verbose=3)

In [34]:
re=grid.cv_results_     #print best parameter after tunning

In [35]:
re

{'mean_fit_time': array([0.14587669, 1.16489758, 0.12651134, 0.88771009, 0.13918061,
        0.93640275, 0.72150083, 9.27946844, 0.534094  , 5.28360853,
        0.35029006, 6.93662405]),
 'std_fit_time': array([0.02624255, 0.1630602 , 0.01610733, 0.05358792, 0.03031822,
        0.12948983, 0.13874801, 0.17836785, 0.11723373, 0.25108781,
        0.01955617, 0.26895673]),
 'mean_score_time': array([0.0138823 , 0.0549078 , 0.02086844, 0.05100579, 0.01903596,
        0.04634538, 0.01906023, 0.03044624, 0.01160617, 0.08371749,
        0.01544466, 0.04932179]),
 'std_score_time': array([0.00086422, 0.00992333, 0.00804736, 0.00787376, 0.00530232,
        0.00413127, 0.01018915, 0.01850276, 0.00605621, 0.07471276,
        0.01055919, 0.05099355]),
 'param_criterion': masked_array(data=['mse', 'mse', 'mse', 'mse', 'mse', 'mse', 'mae', 'mae',
                    'mae', 'mae', 'mae', 'mae'],
              mask=[False, False, False, False, False, False, False, False,
                    False, Fal

In [36]:
#Evaluate the Model: Extract the best parameters and test the model.
best_params = grid.best_params_
best_model = grid.best_estimator_

print("Best Parameters:", best_params)
print("Best Model Performance on Test Set:", best_model.score(independent, dependent))

Best Parameters: {'criterion': 'mae', 'max_features': 'log2', 'n_estimators': 100}
Best Model Performance on Test Set: 0.9629315494582243


In [37]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
print("The R_score value for best parameter {}:".format(grid.best_params_))

The R_score value for best parameter {'criterion': 'mae', 'max_features': 'log2', 'n_estimators': 100}:


In [38]:
table=pd.DataFrame.from_dict(re)

In [39]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.145877,0.026243,0.013882,0.000864,mse,auto,10,"{'criterion': 'mse', 'max_features': 'auto', '...",-2.704142e+07,-2.989612e+07,-2.299787e+07,-2.986239e+07,-2.732504e+07,-2.742457e+07,2.522060e+06,11
1,1.164898,0.163060,0.054908,0.009923,mse,auto,100,"{'criterion': 'mse', 'max_features': 'auto', '...",-2.430128e+07,-3.096737e+07,-2.214526e+07,-2.879455e+07,-2.625510e+07,-2.649271e+07,3.133057e+06,8
2,0.126511,0.016107,0.020868,0.008047,mse,sqrt,10,"{'criterion': 'mse', 'max_features': 'sqrt', '...",-2.462951e+07,-3.186486e+07,-2.484839e+07,-3.005915e+07,-2.717864e+07,-2.771611e+07,2.854531e+06,12
3,0.887710,0.053588,0.051006,0.007874,mse,sqrt,100,"{'criterion': 'mse', 'max_features': 'sqrt', '...",-2.192946e+07,-2.910714e+07,-2.115249e+07,-2.569792e+07,-2.450975e+07,-2.447935e+07,2.845072e+06,2
4,0.139181,0.030318,0.019036,0.005302,mse,log2,10,"{'criterion': 'mse', 'max_features': 'log2', '...",-2.309921e+07,-3.238988e+07,-2.409511e+07,-2.739915e+07,-2.575689e+07,-2.654805e+07,3.267681e+06,9
5,0.936403,0.129490,0.046345,0.004131,mse,log2,100,"{'criterion': 'mse', 'max_features': 'log2', '...",-2.305106e+07,-2.972929e+07,-2.091838e+07,-2.637981e+07,-2.490630e+07,-2.499697e+07,2.991212e+06,4
6,0.721501,0.138748,0.019060,0.010189,mae,auto,10,"{'criterion': 'mae', 'max_features': 'auto', '...",-2.820786e+07,-3.208996e+07,-2.101002e+07,-2.812748e+07,-2.618774e+07,-2.712461e+07,3.608319e+06,10
7,9.279468,0.178368,0.030446,0.018503,mae,auto,100,"{'criterion': 'mae', 'max_features': 'auto', '...",-2.542039e+07,-3.125343e+07,-2.100865e+07,-2.702353e+07,-2.409432e+07,-2.576006e+07,3.383471e+06,5
8,0.534094,0.117234,0.011606,0.006056,mae,sqrt,10,"{'criterion': 'mae', 'max_features': 'sqrt', '...",-2.281251e+07,-3.106984e+07,-2.176902e+07,-2.896801e+07,-2.479846e+07,-2.588357e+07,3.576298e+06,6
9,5.283609,0.251088,0.083717,0.074713,mae,sqrt,100,"{'criterion': 'mae', 'max_features': 'sqrt', '...",-2.256770e+07,-2.954996e+07,-2.214090e+07,-2.615167e+07,-2.423333e+07,-2.492871e+07,2.708555e+06,3


In [40]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age:22
BMI:27
Children:0
Sex Male 0 or 1:0
Smoker Yes 0 or 1:1


In [41]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
#␣change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[18736.33]


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"
